In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '2'
os.environ['export OPENBLAS_NUM_THREADS']='2'

In [2]:
import torch
import torch.nn as nn
import torch.fft
import numpy as np
from scipy.io import loadmat, savemat
import math
import os
import h5py
import matplotlib.pyplot as plt
from functools import partial
from models.models import MWT2d
from models.utils import train, test, LpLoss, get_filter, UnitGaussianNormalizer

In [3]:
print(torch.__version__)

1.13.1+cu117


In [4]:
torch.manual_seed(0)
np.random.seed(0)

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
def get_initializer(name):
    
    if name == 'xavier_normal':
        init_ = partial(nn.init.xavier_normal_)
    elif name == 'kaiming_uniform':
        init_ = partial(nn.init.kaiming_uniform_)
    elif name == 'kaiming_normal':
        init_ = partial(nn.init.kaiming_normal_)
    return init_

In [8]:
data_path = 'Data/temps_train.npy'
ntrain = 3800
ntest = 616

r = 1
h = int(((64 - 1)/r) + 1)
s = h

dataloader = np.load(data_path)
print(dataloader.shape)
u_data = dataloader.astype(np.float32)

x_train = torch.from_numpy(u_data[:ntrain, ::r,::r, 0])
y_train = torch.from_numpy(u_data[:ntrain, ::r,::r, 1])

x_test = torch.from_numpy(u_data[-ntest:, ::r,::r, 0])
y_test = torch.from_numpy(u_data[-ntest:, ::r,::r, 1])

(4416, 64, 64, 2)


In [9]:
x_normalizer = UnitGaussianNormalizer(x_train)
x_train = x_normalizer.encode(x_train)
x_test = x_normalizer.encode(x_test)

y_normalizer = UnitGaussianNormalizer(y_train)
y_train = y_normalizer.encode(y_train)

grids = []
grids.append(np.linspace(0, 1, s))
grids.append(np.linspace(0, 1, s))
grid = np.vstack([xx.ravel() for xx in np.meshgrid(*grids)]).T
grid = grid.reshape(1,s,s,2)
grid = torch.tensor(grid, dtype=torch.float)
x_train = torch.cat([x_train.reshape(ntrain,s,s,1), grid.repeat(ntrain,1,1,1)], dim=3)
x_test = torch.cat([x_test.reshape(ntest,s,s,1), grid.repeat(ntest,1,1,1)], dim=3)

In [10]:
# # data_path = 'Data/melt64/melting.npy'
# data_path = 'Data/melt64/melting.npy'
# dataloader = np.load(data_path)

# ntrain = 170
# ntest = 30

# r = 1
# h = int(((64 - 1)/r) + 1)
# s = h

In [11]:
batch_size = 20
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=True)

In [12]:
ich = 3
initializer = get_initializer('xavier_normal') # xavier_normal, kaiming_normal, kaiming_uniform

torch.manual_seed(0)
np.random.seed(0)

model = MWT2d(ich, 
            alpha = 12,
            c = 4,
            k = 4, 
            base = 'legendre', # 'chebyshev'
            nCZ = 4,
            L = 0,
            initializer = initializer,
            ).to(device)

learning_rate = 0.001
epochs = 5000
step_size = 100
gamma = 0.5

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)
y_normalizer.cuda()

In [14]:
train_loss = []
test_loss = []
for epoch in range(1, epochs+1):
    train_l2 = train(model, train_loader, optimizer, epoch, device,
        lossFn = myloss, lr_schedule = scheduler,
        post_proc = y_normalizer.decode)
    train_loss.append(train_l2)
    test_l2 = test(model, test_loader, device, lossFn=myloss, post_proc=y_normalizer.decode)
    print(f'epoch: {epoch}, train l2 = {train_l2}, test l2 = {test_l2}')
    test_loss.append(test_l2)
        
    if epoch%100 == 0:
        PATH = 'NS_models/temps_new/temps_new{}.pt'.format(epoch)
        torch.save({'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': myloss}, PATH)
        
        np.save('visual/train_loss_temps_much_data.npy', train_loss)
        np.save('visual/test_loss_temps_much_data.npy', test_loss)

epoch: 1, train l2 = 0.14023416360742166, test l2 = 0.08355708768615475
epoch: 2, train l2 = 0.07588366008118579, test l2 = 0.08258027438219491
epoch: 3, train l2 = 0.06464024584544333, test l2 = 0.07354230617547965
epoch: 4, train l2 = 0.06070535225303549, test l2 = 0.05038590119643645
epoch: 5, train l2 = 0.056148563905766134, test l2 = 0.061369071436392794
epoch: 6, train l2 = 0.05577198911654322, test l2 = 0.046795936277160395
epoch: 7, train l2 = 0.053693953579977936, test l2 = 0.0494527192665385
epoch: 8, train l2 = 0.049702709254465606, test l2 = 0.044418110766194084
epoch: 9, train l2 = 0.051230339737314924, test l2 = 0.04260125698207261
epoch: 10, train l2 = 0.05354311256032241, test l2 = 0.04556868273716468
epoch: 11, train l2 = 0.05150753734927428, test l2 = 0.04985572088074375
epoch: 12, train l2 = 0.04855660590686296, test l2 = 0.04056264827777813
epoch: 13, train l2 = 0.04494076333547893, test l2 = 0.04137165631566729
epoch: 14, train l2 = 0.0461248777414623, test l2 = 0.

epoch: 113, train l2 = 0.037246450891620236, test l2 = 0.03848061942821973
epoch: 114, train l2 = 0.03815368242953953, test l2 = 0.037260713660484786
epoch: 115, train l2 = 0.03734507581905315, test l2 = 0.04068565146102534
epoch: 116, train l2 = 0.03704279485501741, test l2 = 0.038250515406781975
epoch: 117, train l2 = 0.036991114506596014, test l2 = 0.03708292031055921
epoch: 118, train l2 = 0.03705451263408912, test l2 = 0.036839434175522295
epoch: 119, train l2 = 0.03722797536536267, test l2 = 0.03809897712879367
epoch: 120, train l2 = 0.03759795093222668, test l2 = 0.037386746665874084
epoch: 121, train l2 = 0.03720612743967458, test l2 = 0.03881070250040525
epoch: 122, train l2 = 0.03857629087410475, test l2 = 0.03857764563003144
epoch: 123, train l2 = 0.03796795824640676, test l2 = 0.03856435451995243
epoch: 124, train l2 = 0.03720957622716301, test l2 = 0.03750046400667785
epoch: 125, train l2 = 0.0382795049799116, test l2 = 0.04463109519187506
epoch: 126, train l2 = 0.03773752

epoch: 223, train l2 = 0.035972677265342916, test l2 = 0.03701653563744062
epoch: 224, train l2 = 0.036101119204571375, test l2 = 0.03686819380366957
epoch: 225, train l2 = 0.036088741531497555, test l2 = 0.03848907728860905
epoch: 226, train l2 = 0.0360955617537624, test l2 = 0.036863112217420106
epoch: 227, train l2 = 0.03580434794488706, test l2 = 0.037162836979735984
epoch: 228, train l2 = 0.03611980177854237, test l2 = 0.03740678505076991
epoch: 229, train l2 = 0.03605048534117247, test l2 = 0.03670611503449353
epoch: 230, train l2 = 0.03597095040114302, test l2 = 0.03777651227526851
epoch: 231, train l2 = 0.036600459602318315, test l2 = 0.04171906556788977
epoch: 232, train l2 = 0.03680959782318065, test l2 = 0.03798094382146736
epoch: 233, train l2 = 0.036143892893665715, test l2 = 0.03723554371239303
epoch: 234, train l2 = 0.035933170977391696, test l2 = 0.03716547105025936
epoch: 235, train l2 = 0.03582658292431581, test l2 = 0.036599120052603934
epoch: 236, train l2 = 0.03611

epoch: 333, train l2 = 0.03532417889488371, test l2 = 0.036339994955372504
epoch: 334, train l2 = 0.035412205183192304, test l2 = 0.037627109749750656
epoch: 335, train l2 = 0.03539281498444708, test l2 = 0.036454328275346136
epoch: 336, train l2 = 0.03578735895062748, test l2 = 0.03665118064586218
epoch: 337, train l2 = 0.03549812782751886, test l2 = 0.03635101184829489
epoch: 338, train l2 = 0.03518662568769957, test l2 = 0.036515504121780396
epoch: 339, train l2 = 0.03531960056800591, test l2 = 0.037301430164219496
epoch: 340, train l2 = 0.03552633355322637, test l2 = 0.037539666155716044
epoch: 341, train l2 = 0.03532435137190317, test l2 = 0.03667046400633725
epoch: 342, train l2 = 0.03528787040396741, test l2 = 0.03647717852871139
epoch: 343, train l2 = 0.0357334099396279, test l2 = 0.03651296705394596
epoch: 344, train l2 = 0.035812527262850814, test l2 = 0.0368668031576392
epoch: 345, train l2 = 0.035530934631824496, test l2 = 0.037555757764872015
epoch: 346, train l2 = 0.03535

epoch: 444, train l2 = 0.035065390781352394, test l2 = 0.03655675111652969
epoch: 445, train l2 = 0.035108587749694525, test l2 = 0.03700052559762806
epoch: 446, train l2 = 0.035092600952637824, test l2 = 0.03677010332996195
epoch: 447, train l2 = 0.03518327800851119, test l2 = 0.03697612717167124
epoch: 448, train l2 = 0.03513821691274643, test l2 = 0.036348885384859975
epoch: 449, train l2 = 0.03506395138408008, test l2 = 0.03663142131907599
epoch: 450, train l2 = 0.03509721979498863, test l2 = 0.036361299406785466
epoch: 451, train l2 = 0.03509780397540645, test l2 = 0.0367640746007492
epoch: 452, train l2 = 0.0350877557694912, test l2 = 0.03637914224104448
epoch: 453, train l2 = 0.03510356501529091, test l2 = 0.03641752347156599
epoch: 454, train l2 = 0.03505546607469258, test l2 = 0.03646830943497745
epoch: 455, train l2 = 0.03502698345403922, test l2 = 0.03635831511059365
epoch: 456, train l2 = 0.03511263346201495, test l2 = 0.037390356714075264
epoch: 457, train l2 = 0.035216968

...

epoch: 2088, train l2 = 0.0347156496424424, test l2 = 0.036348958342493354
epoch: 2089, train l2 = 0.034715649211093, test l2 = 0.03634896429328175
epoch: 2090, train l2 = 0.03471564879542903, test l2 = 0.036348956455658006
epoch: 2091, train l2 = 0.034715647438639086, test l2 = 0.036348960180948305
epoch: 2092, train l2 = 0.034715649195407566, test l2 = 0.036348958342493354
epoch: 2093, train l2 = 0.034715649195407566, test l2 = 0.0363489580038306
epoch: 2094, train l2 = 0.034715647791561326, test l2 = 0.03634895776192863
epoch: 2095, train l2 = 0.034715649344419175, test l2 = 0.03634895423015991
epoch: 2096, train l2 = 0.03471564887385619, test l2 = 0.03634895669755998
epoch: 2097, train l2 = 0.034715649375790046, test l2 = 0.03634896110017578
epoch: 2098, train l2 = 0.03471564769744873, test l2 = 0.03634895819735218
epoch: 2099, train l2 = 0.034715649093452254, test l2 = 0.0363489550042462
epoch: 2100, train l2 = 0.03471564982282488, test l2 = 0.03634895171437945
epoch: 2101, train 

epoch: 2197, train l2 = 0.034715649203250286, test l2 = 0.03634895655241879
epoch: 2198, train l2 = 0.034715649618914254, test l2 = 0.03634895597185407
epoch: 2199, train l2 = 0.03471564932873374, test l2 = 0.03634896114855617
epoch: 2200, train l2 = 0.03471565017574712, test l2 = 0.0363489609550346
epoch: 2201, train l2 = 0.034715649289520166, test l2 = 0.03634895848763454
epoch: 2202, train l2 = 0.034715649195407566, test l2 = 0.0363489587779169
epoch: 2203, train l2 = 0.03471564965028512, test l2 = 0.03634895814897178
epoch: 2204, train l2 = 0.03471564924246386, test l2 = 0.03634895519776778
epoch: 2205, train l2 = 0.034715648669945566, test l2 = 0.03634895964876398
epoch: 2206, train l2 = 0.034715648936597926, test l2 = 0.03634895781030903
epoch: 2207, train l2 = 0.03471565027770243, test l2 = 0.03634896158397972
epoch: 2208, train l2 = 0.03471565061493924, test l2 = 0.0363489572297443
epoch: 2209, train l2 = 0.034715648850328046, test l2 = 0.03634895539128935
epoch: 2210, train l2

epoch: 2306, train l2 = 0.03471564880327174, test l2 = 0.036348963228913095
epoch: 2307, train l2 = 0.034715649375790046, test l2 = 0.03634895819735218
epoch: 2308, train l2 = 0.034715648748372734, test l2 = 0.036348958294112964
epoch: 2309, train l2 = 0.034715649085609535, test l2 = 0.0363489580038306
epoch: 2310, train l2 = 0.034715648575832966, test l2 = 0.03634895839087375
epoch: 2311, train l2 = 0.034715649085609535, test l2 = 0.03634895572995211
epoch: 2312, train l2 = 0.03471564818369715, test l2 = 0.036348961003414997
epoch: 2313, train l2 = 0.034715648324866046, test l2 = 0.03634895824573257
epoch: 2314, train l2 = 0.034715649446374494, test l2 = 0.036348960761513026
epoch: 2315, train l2 = 0.034715648567990254, test l2 = 0.0363489557783325
epoch: 2316, train l2 = 0.0347156496894987, test l2 = 0.036348957665167846
epoch: 2317, train l2 = 0.0347156501208481, test l2 = 0.036348952778748105
epoch: 2318, train l2 = 0.034715649195407566, test l2 = 0.03634895626213643
epoch: 2319, t

epoch: 2415, train l2 = 0.03471564941500362, test l2 = 0.0363489565040384
epoch: 2416, train l2 = 0.034715648677788286, test l2 = 0.036348957955450206
epoch: 2417, train l2 = 0.034715649258149295, test l2 = 0.03634895926172083
epoch: 2418, train l2 = 0.03471564852877667, test l2 = 0.036348957132983514
epoch: 2419, train l2 = 0.03471564885817076, test l2 = 0.03634895993904634
epoch: 2420, train l2 = 0.03471564846603494, test l2 = 0.036348956842701154
epoch: 2421, train l2 = 0.034715648936597926, test l2 = 0.03634895606861486
epoch: 2422, train l2 = 0.03471564904639596, test l2 = 0.036348961390458144
epoch: 2423, train l2 = 0.034715648481720374, test l2 = 0.03634895495586581
epoch: 2424, train l2 = 0.034715649093452254, test l2 = 0.036348955875093285
epoch: 2425, train l2 = 0.03471564918756485, test l2 = 0.036348954326920695
epoch: 2426, train l2 = 0.034715649281677446, test l2 = 0.03634895824573257
epoch: 2427, train l2 = 0.03471564869347372, test l2 = 0.03634896182588169
epoch: 2428, t

KeyboardInterrupt: 

In [ ]:
# PATH = 'NS_models/melt/melt.pt'

# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

# model.eval()

In [ ]:
pred = torch.zeros(y_test.shape)
index = 0
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=1, shuffle=False)
with torch.no_grad():
    for x, y in test_loader:
        test_l2 = 0
        x, y = x.cuda(), y.cuda()
        out = model(x)
        pred[index] = out

        test_l2 += myloss(out.view(1, -1), y.view(1, -1)).item()
        print(index, test_l2)
        index = index + 1

In [ ]:
total_loss = 0.
predictions = []
post_proc=y_normalizer.decode

with torch.no_grad():
    for data, target in test_loader:
        bs = len(data)

        data, target = data.to(device), target.to(device)
        output = model(data)
        output = post_proc(output)

        loss = myloss(output.view(bs, -1), target.view(bs, -1))
        predictions.extend(output.cpu().data.numpy())
        total_loss += loss.sum().item()

predictions = torch.Tensor(predictions)
predictions = torch.reshape(predictions, y_test.shape)

In [ ]:
print('x_test.mean.shape', x_test[29, :, :, 0])
print('test_u[0].shape', y_test.shape)
print('predictions', predictions.shape)
print('total loss = ', total_loss)

In [ ]:
def plot_loss(initial, prediction, test, index):
    initial = initial[index]
    test = test[index]
    prediction = prediction[index]
    
    loss = abs(torch.sub(test, prediction))
#     test0 = test[index][:, :, 0].T
#     prediction0 = prediction[index][:, :, 0].T
#     loss0 = abs(torch.sub(test0, prediction0))
#     print(test.shape)
#     test39 = test[index][:, :, 39].T
#     prediction39 = prediction[index][:, :, 39].T
#     loss39 = abs(torch.sub(test39, prediction39))
# #     print(test)
# #     print(prediction)
# #     print(loss)
    fig, axs = plt.subplots(2, 2, figsize=(20, 20))

    cp1 = axs[0, 0].matshow(test)
    axs[0, 0].set_title('True velocity field', fontsize=20)
    axs[0, 0].xaxis.set_tick_params(labelsize=18)
    axs[0, 0].yaxis.set_tick_params(labelsize=18)
    cp1 = fig.colorbar(cp1)
    cp1.ax.tick_params(labelsize=18)
    
    cp2 = axs[0, 1].matshow(prediction)
    axs[0, 1].set_title('Predicted velocity field', fontsize=20)
    axs[0, 1].xaxis.set_tick_params(labelsize=18)
    axs[0, 1].yaxis.set_tick_params(labelsize=18)
    cp2 = fig.colorbar(cp2)
    cp2.ax.tick_params(labelsize=18)
    
    cp3 = axs[1, 0].matshow(loss/100)
    axs[1, 0].set_title('Absolute difference between model \n prediction and true value', fontsize=20)
    axs[1, 0].xaxis.set_tick_params(labelsize=18)
    axs[1, 0].yaxis.set_tick_params(labelsize=18)
    cp3 = fig.colorbar(cp3)
    cp3.ax.tick_params(labelsize=18)
    
    
    train = np.load('visual/ice_train_loss_xyu.npy')
    test = np.load('visual/ice_test_loss_xyu.npy')
    axs[1, 1].plot(train[100:1100], label = 'Train loss')
    axs[1, 1].plot(test[100:1100], label = 'Test loss')
    axs[1, 1].set_title('End of predicted field', fontsize=20)
    axs[1, 1].xaxis.set_tick_params(labelsize=18)
    axs[1, 1].yaxis.set_tick_params(labelsize=18)
    axs[1, 1].tick_params(labelsize=18)
    axs[1, 1].legend(prop={'size': 18})
    fig.show()

In [ ]:
plot_loss(x_train, predictions, y_test, 0)

In [ ]:
cp1 = plt.matshow(y_train[0])
plt.colorbar(cp1)